# ДЗ 07 часть 2. ML для бизнеса

Домашнее задание (часть 2)

1. [Придумать новые признаки (у нас их всего 5 было) - сделать дополнительный feature engineering](#task01)
2. [Для подходов с одной моделью поварьировать параметры CatBoostClassifier либо даже попробовать другой алгоритм (опционально уже).](#task02)
3. [Для пункта 2 (подходов на основе одной модели) вывести значимость признаков, основываясь на model.estimator.feature_importances_ и model.estimator.feature_names_.](#task03)
4. [Дополнить models_results дополнительными метриками (roc auc, logloss, f1). В конце вывести сводную таблицу всех показателей для всех подходов (как на основе](#task04)

Дополнительно (полезные ссылки):
1. доклад о персональных сообщениях (рассылке) - https://www.youtube.com/watch?v=UNegf9Rgpnw
2. http://people.cs.pitt.edu/~milos/icml_clinicaldata_2012/Papers/Oral_Jaroszewitz_ICML_Clinical_2012.pdf
3. доклад от Ирины Елисовой (МТС) - https://youtu.be/jCUcYiBK03I
4. https://arxiv.org/pdf/1809.04559.pdf

## Задание 1 <a class='anchor' id='task01'></a>

In [277]:
%matplotlib inline

import pandas as pd
#import pandas as pd; pd.set_option('display.max_columns', None)
from sklearn.model_selection import train_test_split

In [278]:
# Чтение данных
df_clients = pd.read_csv('../../lection7/data/clients.csv', index_col='client_id')
df_train = pd.read_csv('../../lection7/data/uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('../../lection7/data/uplift_test.csv', index_col='client_id')

In [279]:
# Извлечение признаков
df_features = df_clients.copy()
df_features['first_issue_time'] = \
    (pd.to_datetime(df_features['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_features['first_redeem_time'] = \
    (pd.to_datetime(df_features['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_features['issue_redeem_delay'] = df_features['first_redeem_time'] \
    - df_features['first_issue_time']
df_features = df_features.drop(['first_issue_date', 'first_redeem_date'], axis=1)

indices_train = df_train.index
indices_test = df_test.index
indices_learn, indices_valid = train_test_split(df_train.index, 
                                                test_size=0.4, 
                                                random_state=12)

In [280]:
df_features.head(3)

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay
client_id,,,,,
000012768d,45,U,1501947648,1.515094e+09,13146559.0
000036f903,72,F,1491832463,1.492951e+09,1118613.0
000048b7a6,68,F,1544880791,NaN,NaN


### Проверим датасет на пропуски:

In [281]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400162 entries, 000012768d to fffff6ce77
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   age                 400162 non-null  int64  
 1   gender              400162 non-null  object 
 2   first_issue_time    400162 non-null  int64  
 3   first_redeem_time   364693 non-null  float64
 4   issue_redeem_delay  364693 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 18.3+ MB


Заполним все пропуски. Их не так много, поэтому если мы ошибёмся, вряд ли это сильно скажется на результатах:

In [283]:
df_features.loc[df_features['first_redeem_time'].isnull(), 'first_redeem_time']=df_features['first_issue_time']
df_features.loc[df_features['issue_redeem_delay'].isnull(), 'issue_redeem_delay']=0

In [285]:
df_features.head()

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay
client_id,,,,,
000012768d,45,U,1501947648,1.515094e+09,13146559.0
000036f903,72,F,1491832463,1.492951e+09,1118613.0
000048b7a6,68,F,1544880791,1.544881e+09,0.0
000073194a,60,F,1495544174,1.511522e+09,15978107.0
00007c7133,67,U,1495469828,1.546277e+09,50806825.0


In [286]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400162 entries, 000012768d to fffff6ce77
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   age                 400162 non-null  int64  
 1   gender              400162 non-null  object 
 2   first_issue_time    400162 non-null  int64  
 3   first_redeem_time   400162 non-null  float64
 4   issue_redeem_delay  400162 non-null  float64
dtypes: float64(2), int64(2), object(1)
memory usage: 28.3+ MB


In [287]:
# прикрепим обратно столбцы с датами в формате даты, впоследствии это будет нужно
df_features['1st_issue_time']=df_clients['first_issue_date']
df_features['1st_redeem_time']=df_clients['first_redeem_date']
df_features['1st_issue_time'] = pd.to_datetime(df_features['1st_issue_time'])
df_features['1st_redeem_time'] = pd.to_datetime(df_features['1st_redeem_time'])

In [288]:
df_features.loc[df_features['1st_redeem_time'].isnull(), '1st_redeem_time']=df_features['1st_issue_time']

In [289]:
df_features.head()

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,1st_issue_time,1st_redeem_time
client_id,,,,,,,
000012768d,45,U,1501947648,1.515094e+09,13146559.0,2017-08-05 15:40:48,2018-01-04 19:30:07
000036f903,72,F,1491832463,1.492951e+09,1118613.0,2017-04-10 13:54:23,2017-04-23 12:37:56
000048b7a6,68,F,1544880791,1.544881e+09,0.0,2018-12-15 13:33:11,2018-12-15 13:33:11
000073194a,60,F,1495544174,1.511522e+09,15978107.0,2017-05-23 12:56:14,2017-11-24 11:18:01
00007c7133,67,U,1495469828,1.546277e+09,50806825.0,2017-05-22 16:17:08,2018-12-31 17:17:33


### Проверим датасет на выбросы

In [290]:
df_features.describe()

,age,first_issue_time,first_redeem_time,issue_redeem_delay
count,400162.000000,4.001620e+05,4.001620e+05,4.001620e+05
mean,46.488112,1.515519e+09,1.529726e+09,1.420698e+07
std,43.871218,1.759150e+07,1.886355e+07,1.363827e+07
min,-7491.000000,1.491330e+09,1.491817e+09,-3.701689e+06
25%,34.000000,1.500667e+09,1.513953e+09,3.974716e+06
50%,45.000000,1.510845e+09,1.528320e+09,1.009531e+07
75%,59.000000,1.529518e+09,1.546265e+09,2.045765e+07
max,1901.000000,1.552687e+09,1.574212e+09,8.198520e+07


* затруднительно определить выбросы в датах
* видны аномалии (выбросы) в признаке 'age' (возраст)

In [291]:
# проверим, есть ли даты до 2015 года:
df_features[df_features['first_issue_time']<1437984480]

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,1st_issue_time,1st_redeem_time
client_id,,,,,,,


In [292]:
# проверим, есть ли даты после 2019 года:
df_features[df_features['first_issue_time']>1564214880]

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,1st_issue_time,1st_redeem_time
client_id,,,,,,,


* дат до 2015 и после 2019 года в датасете нет

#### обработка выбросов в 'age':

будем считать выбросами значения ниже 14, и выше 90, посмотрим, сколько таких значений

In [293]:
to_drop = df_features.loc[(df_features['age']<14)|(df_features['age']>90)]
to_drop.shape[0]

1869

* Эти выбросные данные мы тоже заменим средним значением

In [294]:
df_features.loc[(df_features['age']<14)|(df_features['age']>90), 'age']=df_features['age'].mean()

* **итак, мы избавились от выбросов**

In [295]:
df_features.describe()

,age,first_issue_time,first_redeem_time,issue_redeem_delay
count,400162.000000,4.001620e+05,4.001620e+05,4.001620e+05
mean,46.408935,1.515519e+09,1.529726e+09,1.420698e+07
std,15.824880,1.759150e+07,1.886355e+07,1.363827e+07
min,14.000000,1.491330e+09,1.491817e+09,-3.701689e+06
25%,34.000000,1.500667e+09,1.513953e+09,3.974716e+06
50%,45.000000,1.510845e+09,1.528320e+09,1.009531e+07
75%,59.000000,1.529518e+09,1.546265e+09,2.045765e+07
max,90.000000,1.552687e+09,1.574212e+09,8.198520e+07


In [296]:
df_features.head(3)

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,1st_issue_time,1st_redeem_time
client_id,,,,,,,
000012768d,45.0,U,1501947648,1.515094e+09,13146559.0,2017-08-05 15:40:48,2018-01-04 19:30:07
000036f903,72.0,F,1491832463,1.492951e+09,1118613.0,2017-04-10 13:54:23,2017-04-23 12:37:56
000048b7a6,68.0,F,1544880791,1.544881e+09,0.0,2018-12-15 13:33:11,2018-12-15 13:33:11


### Feature engineering

Создадим несколько новых признаков: признаки "Пенсионер yes/no", "день недели", "месяц", "холодное время года yes/no","выходной yes/no", "праздничный день yes/no", возможно, попробуем взять квантили по возрасту, чтобы создать группы

#### Признак 'retired', бинарный. Относится ли человек по возрасту к пенсионеру

In [297]:
df_features['retired']=0
df_features.loc[((df_features['age']>55) & (df_features['gender']=="F"))|((df_features['age']>60)&(df_features['gender']=="M")), 'retired']=1

In [298]:
df_features['retired'].sum()

64955

#### Признаки "dayofweek_i", "dayofweek_r"

- соотвественно, день недели даты выпуска и даты активации (issue and redeem)

In [299]:
df_features['dayofweek_i']=df_features['1st_issue_time'].dt.dayofweek
df_features['dayofweek_r']=df_features['1st_redeem_time'].dt.dayofweek
df_features['month_i']=df_features['1st_redeem_time'].dt.month
df_features['month_r']=df_features['1st_redeem_time'].dt.month
df_features['daymonth_i']=df_features['1st_redeem_time'].dt.day
df_features['daymonth_r']=df_features['1st_redeem_time'].dt.day

In [300]:
df_features.head()

,age,gender,first_issue_time,first_redeem_time,issue_redeem_delay,1st_issue_time,1st_redeem_time,retired,dayofweek_i,dayofweek_r,month_i,month_r,daymonth_i,daymonth_r
client_id,,,,,,,,,,,,,,
000012768d,45.0,U,1501947648,1.515094e+09,13146559.0,2017-08-05 15:40:48,2018-01-04 19:30:07,0,5,3,1,1,4,4
000036f903,72.0,F,1491832463,1.492951e+09,1118613.0,2017-04-10 13:54:23,2017-04-23 12:37:56,1,0,6,4,4,23,23
000048b7a6,68.0,F,1544880791,1.544881e+09,0.0,2018-12-15 13:33:11,2018-12-15 13:33:11,1,5,5,12,12,15,15
000073194a,60.0,F,1495544174,1.511522e+09,15978107.0,2017-05-23 12:56:14,2017-11-24 11:18:01,1,1,4,11,11,24,24
00007c7133,67.0,U,1495469828,1.546277e+09,50806825.0,2017-05-22 16:17:08,2018-12-31 17:17:33,0,0,0,12,12,31,31


* пока хватит

# Задание 2 <a class='anchor' id='task02'></a>

Для подходов с одной моделью поварьировать параметры CatBoostClassifier либо даже попробовать другой алгоритм (опционально уже).

In [340]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [341]:
from catboost import CatBoostClassifier

In [306]:
X_train = df_features.loc[indices_learn, :]
y_train = df_train.loc[indices_learn, 'target']
treat_train = df_train.loc[indices_learn, 'treatment_flg']

X_val = df_features.loc[indices_valid, :]
y_val = df_train.loc[indices_valid, 'target']
treat_val =  df_train.loc[indices_valid, 'treatment_flg']

X_train_full = df_features.loc[indices_train, :]
y_train_full = df_train.loc[:, 'target']
treat_train_full = df_train.loc[:, 'treatment_flg']

X_test = df_features.loc[indices_test, :]

cat_features = ['gender', 'retired']

models_results = {
    'approach': [],
    'uplift@30%': []
}

In [346]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
def evaluate_results(y_test, y_predict):
    #print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    #print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    #print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    #print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    #print("precision: %.2f%%" % (prc * 100.0)) 
    loglss = log_loss(y_test, y_predict)
    
    item = {'f1':f1, 'roc':roc, 'rec':rec,'prc':prc, 'log_loss':loglss}
    
    return item

In [358]:
%%time
iterations_catb = [2,5,10]#,20,40]#,5,10,20,40,100]
depths = [2, 5, 10]#, 16]#,5,10,16]
results_list= []
for iters in iterations_catb:
    for depth in depths:
        
        sm = SoloModel(CatBoostClassifier(iterations=iters, max_depth = depth, thread_count=2, random_state=42, silent=True))
        
        sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

        uplift_sm = sm.predict(X_val)

        sm_score = uplift_at_k(y_true=y_val, uplift=uplift_sm, treatment=treat_val, strategy='by_group', k=0.3)
        
        
        
#         models_results['approach'].append('SoloModel')
#         models_results['uplift@30%'].append(sm_score)
        item = {'approach':'SoloModel', 'uplift@30%':sm_score, 'iterations':iters, 'depth':depth}

#         item = evaluate_results(y_val,treat_val)
#         item['approach']='SoloModel'
#         item['uplift@30%']=sm_score
        item['iterations']=iters
        item['depth']=depth
        print(item)
        results_list.append(item)
        #print(item)
        # Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
        sm_trmnt_preds = sm.trmnt_preds_
        # И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
        sm_ctrl_preds = sm.ctrl_preds_


{'approach': 'SoloModel', 'uplift@30%': 0.029434305697257912, 'iterations': 2, 'depth': 2}
{'approach': 'SoloModel', 'uplift@30%': 0.029434305697257912, 'iterations': 2, 'depth': 5}
{'approach': 'SoloModel', 'uplift@30%': 0.05849121823998693, 'iterations': 2, 'depth': 10}
{'approach': 'SoloModel', 'uplift@30%': 0.029156005841131316, 'iterations': 5, 'depth': 2}
{'approach': 'SoloModel', 'uplift@30%': 0.06633502831885696, 'iterations': 5, 'depth': 5}
{'approach': 'SoloModel', 'uplift@30%': 0.054165338272197716, 'iterations': 5, 'depth': 10}
{'approach': 'SoloModel', 'uplift@30%': 0.025973940503533854, 'iterations': 10, 'depth': 2}
{'approach': 'SoloModel', 'uplift@30%': 0.053875292173889866, 'iterations': 10, 'depth': 5}
{'approach': 'SoloModel', 'uplift@30%': 0.032620508600068154, 'iterations': 10, 'depth': 10}
CPU times: user 17.2 s, sys: 907 ms, total: 18.1 s
Wall time: 14.8 s


In [359]:
res = pd.DataFrame(results_list)
(res.sort_values('uplift@30%', ascending=False)).head()

,approach,uplift@30%,iterations,depth
4,SoloModel,0.066335,5,5
2,SoloModel,0.058491,2,10
5,SoloModel,0.054165,5,10
7,SoloModel,0.053875,10,5
8,SoloModel,0.032621,10,10


## Задание 3 <a class='anchor' id='task03'></a>

Для пункта 2 (подходов на основе одной модели) вывести значимость признаков, основываясь на model.estimator.feature_importances_ и model.estimator.feature_names_.

In [323]:
sm_fi = pd.DataFrame({
    'feature_name': sm.estimator.feature_names_,
    'feature_score': sm.estimator.feature_importances_
}).sort_values('feature_score', ascending=False).reset_index(drop=True)

sm_fi

,feature_name,feature_score
0,dayofweek_r,11.364703
1,age,11.297192
2,dayofweek_i,11.224094
3,issue_redeem_delay,9.904308
4,treatment,9.638402
5,gender,8.155788
6,daymonth_i,6.302881
7,daymonth_r,6.081816
8,month_r,5.093425
9,1st_issue_time,4.449793


## Задание 4 <a class='anchor' id='task04'></a>

Дополнить models_results дополнительными метриками (roc auc, logloss, f1). В конце вывести сводную таблицу всех показателей для всех подходов (как на основе

In [381]:
%%time
iterations_catb = [2,5,10]#,20,40]#,5,10,20,40,100]
depths = [2, 5, 10]#, 16]#,5,10,16]
results_list= []
for iters in iterations_catb:
    for depth in depths:
        
        sm1 = SoloModel(CatBoostClassifier(iterations=iters, max_depth = depth, thread_count=2, random_state=42, silent=True))
        sm2 = CatBoostClassifier(iterations=iters, max_depth = depth, thread_count=2, random_state=42, silent=True)
        
        sm = sm1.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})
        sm2 = sm2.fit(X_train, y_train, cat_features)
        
        uplift_sm = sm.predict(X_val)
        y_pred = sm2.predict(X_val)
        
        sm_score = uplift_at_k(y_true=y_val, uplift=uplift_sm, treatment=treat_val, strategy='by_group', k=0.3)
        
        item = evaluate_results(y_val,y_pred)
        item['approach']='SoloModel'
        item['uplift@30%']=sm_score
        item['iterations']=iters
        item['depth']=depth
        #print(item)
        results_list.append(item)
        #print(item)
        # Получим условные вероятности выполнения целевого действия при взаимодействии для каждого объекта
        sm_trmnt_preds = sm.trmnt_preds_
        # И условные вероятности выполнения целевого действия без взаимодействия для каждого объекта
        sm_ctrl_preds = sm.ctrl_preds_


{'f1': 0.7683848124398323, 'roc': 0.5485910439941295, 'rec': 0.9476586239510595, 'prc': 0.6461492021240104, 'log_loss': 12.25475508235405, 'approach': 'SoloModel', 'uplift@30%': 0.029434305697257912, 'iterations': 2, 'depth': 2}
{'f1': 0.7655554078308848, 'roc': 0.5580940244213808, 'rec': 0.9269917292173948, 'prc': 0.6520077564365685, 'log_loss': 12.178772946766005, 'approach': 'SoloModel', 'uplift@30%': 0.029434305697257912, 'iterations': 2, 'depth': 5}
{'f1': 0.7648907512720743, 'roc': 0.557462925253249, 'rec': 0.9256635743464874, 'prc': 0.6517008344785572, 'log_loss': 12.206398422796793, 'approach': 'SoloModel', 'uplift@30%': 0.05849121823998693, 'iterations': 2, 'depth': 10}
{'f1': 0.7683848124398323, 'roc': 0.5485910439941295, 'rec': 0.9476586239510595, 'prc': 0.6461492021240104, 'log_loss': 12.25475508235405, 'approach': 'SoloModel', 'uplift@30%': 0.029156005841131316, 'iterations': 5, 'depth': 2}
{'f1': 0.7653941142719344, 'roc': 0.5583868897914838, 'rec': 0.92612641619544, 'prc

In [383]:
res = pd.DataFrame(results_list)
(res.sort_values('uplift@30%', ascending=False)).head()

,f1,roc,rec,prc,log_loss,approach,uplift@30%,iterations,depth
4,0.765394,0.558387,0.926126,0.652202,12.178341,SoloModel,0.066335,5,5
2,0.764891,0.557463,0.925664,0.651701,12.206398,SoloModel,0.058491,2,10
5,0.765524,0.557336,0.927817,0.651555,12.191723,SoloModel,0.054165,5,10
7,0.765065,0.559008,0.924335,0.652615,12.177045,SoloModel,0.053875,10,5
8,0.764367,0.559075,0.922061,0.652734,12.194310,SoloModel,0.032621,10,10
